# Vision Edge-AI와 3D Depth Camera

3D Depth Camera는 공간의 깊이 정보를 캡처하여 3차원 데이터로 변환하는 장치입니다. 이러한 카메라는 다양한 응용 분야에서 사용되며, 컴퓨터 비전, 로봇 공학, 증강현실(AR), 가상현실(VR), 의료 이미지 처리 등에서 중요합니다. 본 강의에서는 3D Depth Camera의 원리, 기술, 응용 사례, 그리고 DeepStream을 활용할 수 있는 방법에 대해 설명합니다.

## 03-2. 3D Depth Camera

### 3D Depth Camera의 정의
3D Depth Camera는 객체나 장면의 깊이 정보를 캡처할 수 있는 장치로, 카메라가 각 픽셀의 거리 값을 캡처하여 3차원 공간에서 물체를 인식하거나 측정할 수 있도록 합니다.

### 3D Depth Camera의 원리
1. **스테레오 비전 (Stereo Vision)**:
    * 두 개의 카메라로 물체를 촬영한 이미지를 비교하여 깊이를 계산합니다.
    * 사람의 시각 시스템과 유사한 방식입니다.

2. **구조광 (Structured Light)**:
    * 특정 패턴의 빛을 물체에 투사하고, 왜곡된 패턴을 분석하여 깊이를 측정합니다.
    * 대표적인 예: Microsoft Kinect

3. **비행시간 (Time of Flight, ToF)**:
    * 빛이 물체에 반사되어 돌아오는 시간을 측정하여 깊이를 계산합니다.
    * 정확하고 빠른 깊이 측정이 가능합니다.

4. **LiDAR (Light Detection and Ranging)**:
    * 레이저를 사용해 주변 환경을 스캔하여 깊이를 측정합니다.
    * 자율주행차에 주로 사용됩니다.

### 응용 분야
1. **산업 및 제조**:
    * 로봇 비전: 로봇의 정확한 동작과 위치 결정을 지원.
    * 품질 검사: 제품의 결함 및 크기 측정.
2. **의료**:
    * 3D 스캔: 인체를 스캔하여 맞춤형 의료 장비 제작.
    * 수술 보조: 정확한 수술 계획을 위한 3D 모델 생성.
3. **엔터테인먼트**:
    * 증강현실/가상현실(AR/VR): 몰입형 환경 구축.
    * 게임: 사용자 동작을 추적하여 인터랙션 제공.
4. **도매(Retail)**:
    * 고객 행동 분석: 매장 내 고객의 동선을 추적하고 행동을 분석.
    * 재고 관리: 제품 크기 및 위치를 자동으로 스캔하여 재고를 효율적으로 관리.
    * 무인 매장: 3D 데이터를 활용해 고객과 제품의 상호작용을 자동화.

## 03-3. OAK-D 모듈
- **OAK-D 모듈 소개**: OAK-D(OpenCV AI Kit Depth)는 AI 및 컴퓨터 비전 기능을 갖춘 카메라 모듈로, 실시간으로 깊이 정보를 제공하며 다양한 응용 분야에 적합합니다.

- **주요 특징**:
  1. **내장형 AI**: Myriad X VPU를 탑재하여 온보드 AI 처리 가능.
  2. **스테레오 비전**: 두 개의 모노 카메라로 깊이 정보 계산.
  3. **컬러 카메라**: 고해상도 컬러 카메라로 세부 정보를 캡처.
  4. **플러그 앤 플레이**: USB를 통해 쉽게 연결 및 사용 가능.
  5. **다양한 SDK 지원**: DepthAI 및 OpenCV와 같은 SDK 지원.

- **응용 사례**:
  - 로봇 내비게이션
  - 객체 추적 및 감지
  - 증강 현실 및 가상 현실 애플리케이션

---

# 간단한 예제 실습

## Jetson Orin Nano에서 OAK-D PRO setting (Connection)

### 준비물
- Jetson Orin Nano
- OAK-D PRO Camera
- USB-C 케이블

### 필수 라이브러리 설치
라이브러리 설치
```bash
# 설치되어있는 OpenCV 삭제
$ pip uninstall opencv-python

# 저장소 추가
$ sudo add-apt-repository universe
$ sudo apt update

# 필수 라이브러리 설치
$ sudo apt install -y build-essential cmake git pkg-config libgtk-3-dev libavcodec-dev libavformat-dev libswscale-dev libv4l-dev libxvidcore-dev libx264-dev libjpeg-dev libpng-dev libtiff-dev gfortran openexr libatlas-base-dev python3-dev python3-numpy libtbb2 libtbb-dev libdc1394-dev

# OpenCV 설치
pip install opencv-python
```

OAK-D 관련 드라이버 설치
```bash
$ wget -qO- https://docs.luxonis.com/install_dependencies.sh | bash
$ pip3 install depthai --upgrade
$ pip3 install blobconverter
```
* 드라이버 설치 이후 반드시 OAK-D 모듈 연결 USB를 뺐다가 다시 

In [1]:
import depthai as dai
import cv2

# Pipeline 생성
pipeline = dai.Pipeline()

# 카메라 노드 추가
cam_rgb = pipeline.create(dai.node.ColorCamera)
xout_video = pipeline.create(dai.node.XLinkOut)
xout_video.setStreamName("video")

cam_rgb.video.link(xout_video.input)

# Pipeline 실행
with dai.Device(pipeline) as device:
    video_queue = device.getOutputQueue(name="video", maxSize=1, blocking=False)

    while True:
        video_frame = video_queue.get()  # 프레임 가져오기
        frame = video_frame.getCvFrame()
        cv2.imshow("Video", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()

## OAK-D 모듈을 활용한 ObjectDetection

### 필요 모델 다운로드

In [1]:
import blobconverter

# MobileNet-SSD 모델 다운로드 및 Blob 변환
blob_path = blobconverter.from_zoo(
    name="mobilenet-ssd",  # Model name from DepthAI Zoo
    shaves=6,             # Number of shaves for Myriad X
    output_dir="models"   # Save directory for the blob file
)

print(f"Blob file saved at: {blob_path}")

[==================================================]
Done
Blob file saved at: models/mobilenet-ssd_openvino_2022.1_6shave.blob


### Object Detection

In [2]:
import depthai as dai
import cv2

# DepthAI Pipeline 생성
pipeline = dai.Pipeline()

# 카메라 노드 추가 (RGB 카메라)
cam_rgb = pipeline.create(dai.node.ColorCamera)
cam_rgb.setPreviewSize(300, 300)  # 모델의 입력 크기 (300x300은 MobileNet-SSD 크기)
cam_rgb.setInterleaved(False)
cam_rgb.setFps(30)

# 객체 탐지 모델 노드 추가 (MobileNet-SSD)
detection_nn = pipeline.create(dai.node.MobileNetDetectionNetwork)
detection_nn.setBlobPath("./models/mobilenet-ssd_openvino_2022.1_6shave.blob")  # 모델 경로 지정
detection_nn.setConfidenceThreshold(0.8)  # 탐지 최소 신뢰도 설정

# RGB 카메라의 출력을 객체 탐지 네트워크로 연결
cam_rgb.preview.link(detection_nn.input)

# RGB 프레임 출력 노드 추가
xout_video = pipeline.create(dai.node.XLinkOut)
xout_video.setStreamName("video")
cam_rgb.preview.link(xout_video.input)

# 객체 탐지 결과 출력 노드 추가
xout_detections = pipeline.create(dai.node.XLinkOut)
xout_detections.setStreamName("detections")
detection_nn.out.link(xout_detections.input)

# Device에서 Pipeline 실행
with dai.Device(pipeline) as device:
    # 출력 Queue 설정
    video_queue = device.getOutputQueue(name="video", maxSize=4, blocking=False)
    detections_queue = device.getOutputQueue(name="detections", maxSize=4, blocking=False)

    labels = [
        "background", "aeroplane", "bicycle", "bird", "boat",
        "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
        "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
        "sofa", "train", "tvmonitor"
    ]

    while True:
        # 비디오 프레임 가져오기
        video_frame = video_queue.get()
        frame = video_frame.getCvFrame()

        # 객체 탐지 결과 가져오기
        detections = detections_queue.get().detections

        # 탐지 결과 프레임에 표시
        for detection in detections:
            # Bounding box 좌표 변환
            x1 = int(detection.xmin * frame.shape[1])
            y1 = int(detection.ymin * frame.shape[0])
            x2 = int(detection.xmax * frame.shape[1])
            y2 = int(detection.ymax * frame.shape[0])

            # 라벨 이름 가져오기
            label = labels[detection.label]

            # Bounding box와 라벨 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {int(detection.confidence * 100)}%",
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # 결과 화면 표시
        cv2.imshow("Object Detection", frame)

        # 종료 조건
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()


## 객체 탐지 + 깊이 측정 실시간 시각화

In [8]:
import depthai as dai
import cv2
import numpy as np

# DepthAI Pipeline 생성
pipeline = dai.Pipeline()

# RGB 카메라 노드 추가
cam_rgb = pipeline.create(dai.node.ColorCamera)
cam_rgb.setPreviewSize(300, 300)  # 모델 입력 크기
cam_rgb.setInterleaved(False)
cam_rgb.setFps(30)

# 깊이 노드 추가
mono_left = pipeline.create(dai.node.MonoCamera)
mono_right = pipeline.create(dai.node.MonoCamera)
stereo = pipeline.create(dai.node.StereoDepth)

mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
stereo.initialConfig.setConfidenceThreshold(255)

mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)

# 객체 탐지 모델 노드 추가
detection_nn = pipeline.create(dai.node.MobileNetDetectionNetwork)
detection_nn.setBlobPath("./models/mobilenet-ssd_openvino_2022.1_6shave.blob")  # 모델 경로
detection_nn.setConfidenceThreshold(0.9)

cam_rgb.preview.link(detection_nn.input)

# 출력 노드 설정
xout_rgb = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)

xout_depth = pipeline.create(dai.node.XLinkOut)
xout_depth.setStreamName("depth")
stereo.depth.link(xout_depth.input)

xout_nn = pipeline.create(dai.node.XLinkOut)
xout_nn.setStreamName("detections")
detection_nn.out.link(xout_nn.input)

# Device에서 Pipeline 실행
with dai.Device(pipeline) as device:
    # 출력 Queue 설정
    rgb_queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    depth_queue = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    detections_queue = device.getOutputQueue(name="detections", maxSize=4, blocking=False)

    labels = [
        "background", "aeroplane", "bicycle", "bird", "boat",
        "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
        "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
        "sofa", "train", "tvmonitor"
    ]

    while True:
        # RGB 프레임 가져오기
        in_rgb = rgb_queue.get()
        frame = in_rgb.getCvFrame()

        # 깊이 데이터 가져오기
        in_depth = depth_queue.get()
        depth_frame = in_depth.getFrame()  # 단일 채널 깊이 맵
        depth_frame_color = cv2.normalize(depth_frame, None, 0, 255, cv2.NORM_MINMAX)
        depth_frame_color = cv2.applyColorMap(depth_frame_color.astype(np.uint8), cv2.COLORMAP_JET)

        # 객체 탐지 결과 가져오기
        in_detections = detections_queue.get()
        detections = in_detections.detections

        # 객체 탐지 결과를 RGB 프레임과 깊이 맵에 표시
        for detection in detections:
            # Bounding box 좌표 변환
            x1 = int(detection.xmin * frame.shape[1])
            y1 = int(detection.ymin * frame.shape[0])
            x2 = int(detection.xmax * frame.shape[1])
            y2 = int(detection.ymax * frame.shape[0])

            # 라벨 이름 가져오기
            label = labels[detection.label]

            # Bounding box와 라벨 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {int(detection.confidence * 100)}%",
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # 깊이 데이터에서 중심점의 거리 추출
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)
            depth_value = depth_frame[y_center, x_center]

            # 깊이 정보를 화면에 표시
            cv2.putText(frame, f"Depth: {float(depth_value) / 10.0} cm", (x1, y2 + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 결과 화면 표시
        cv2.imshow("RGB Frame", frame)
        cv2.imshow("Depth Map", depth_frame_color)

        # 종료 조건
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()


/tmp/ipykernel_9749/1566211732.py:19: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
/tmp/ipykernel_9749/1566211732.py:20: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)


In [13]:
import depthai as dai
import cv2
import numpy as np

# DepthAI Pipeline 생성
pipeline = dai.Pipeline()

# RGB 카메라 노드 추가
cam_rgb = pipeline.create(dai.node.ColorCamera)
cam_rgb.setPreviewSize(300, 300)  # 모델 입력 크기
cam_rgb.setInterleaved(False)
cam_rgb.setFps(30)

# 깊이 노드 추가
mono_left = pipeline.create(dai.node.MonoCamera)
mono_right = pipeline.create(dai.node.MonoCamera)
stereo = pipeline.create(dai.node.StereoDepth)

mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
stereo.initialConfig.setConfidenceThreshold(255)

mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)

# 객체 탐지 모델 노드 추가
detection_nn = pipeline.create(dai.node.MobileNetDetectionNetwork)
detection_nn.setBlobPath("./models/mobilenet-ssd_openvino_2022.1_6shave.blob")  # 모델 경로
detection_nn.setConfidenceThreshold(0.9)

cam_rgb.preview.link(detection_nn.input)

# 출력 노드 설정
xout_rgb = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)

xout_depth = pipeline.create(dai.node.XLinkOut)
xout_depth.setStreamName("depth")
stereo.depth.link(xout_depth.input)

xout_nn = pipeline.create(dai.node.XLinkOut)
xout_nn.setStreamName("detections")
detection_nn.out.link(xout_nn.input)

# Device에서 Pipeline 실행
with dai.Device(pipeline) as device:
    # 출력 Queue 설정
    rgb_queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    depth_queue = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    detections_queue = device.getOutputQueue(name="detections", maxSize=4, blocking=False)

    labels = [
        "background", "aeroplane", "bicycle", "bird", "boat",
        "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
        "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
        "sofa", "train", "tvmonitor"
    ]

    while True:
        # RGB 프레임 가져오기
        in_rgb = rgb_queue.get()
        frame = in_rgb.getCvFrame()

        # 깊이 데이터 가져오기
        in_depth = depth_queue.get()
        depth_frame = in_depth.getFrame()  # 단일 채널 깊이 맵

        # 객체 탐지 결과 가져오기
        in_detections = detections_queue.get()
        detections = in_detections.detections

        # 객체 탐지 결과를 RGB 프레임에 표시
        for detection in detections:
            # Bounding box 좌표 변환
            x1 = int(detection.xmin * frame.shape[1])
            y1 = int(detection.ymin * frame.shape[0])
            x2 = int(detection.xmax * frame.shape[1])
            y2 = int(detection.ymax * frame.shape[0])

            # 라벨 이름 가져오기
            label = labels[detection.label]

            # Bounding box 중심점 계산
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)

            # 중심점의 깊이 값 가져오기
            depth_value = depth_frame[y_center, x_center]

            # Bounding box와 레이블 출력
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {int(detection.confidence * 100)}% | {depth_value} mm",
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # 결과 화면 표시
        cv2.imshow("Object Detection with Distance", frame)

        # 종료 조건
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()


/tmp/ipykernel_9749/1929540005.py:19: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
/tmp/ipykernel_9749/1929540005.py:20: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
